In [29]:
import numpy as np
import pandas as pd
import logging
from itertools import groupby
from operator import itemgetter

from nltk.internals import deprecated
from nltk.metrics.distance import binary_distance
from nltk.probability import ConditionalFreqDist, FreqDist

In [83]:
# Upload the reliability data as Pandas DataFrame "Reliability"

Reliability = pd.read_csv('Reliability_study.csv', ';', usecols=['text','X_1','X_2','X_3', 'G_1', 'G_2', 'G_3'])

# ISSUE: what about nan value? I think it should be expluded !

# Merge coder's multiple annotations into single columns with tuples of annotations as values 

# Auxiliary functions !! I NEED TO ABSTRACT from X_1, etc.!! 
def create_combined_X(row):
    return (row['X_1'], row['X_2'], row['X_3'])

def create_combined_G(row):
    return (row['G_1'], row['G_2'], row['G_3'])

Reliability['X'] = Reliability.apply(create_combined_X, axis=1)
Reliability['G'] = Reliability.apply(create_combined_G, axis=1)


#print(Reliability['X'], Reliability['G'])

/var/folders/lr/zlwmj0rj26b912f9hjfxqj9c0000gn/T/ipykernel_1855/1939549919.py:3: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only.
  Reliability = pd.read_csv('Reliability_study.csv', ';', usecols=['text','X_1','X_2','X_3', 'G_1', 'G_2', 'G_3'])


In [84]:
# Import MASI distance for set-valued items from NLTK Python library

from nltk.metrics import masi_distance

# Check MASI distances per each item annotated: seems to work!

#for i in range(len(Reliability)):
#    x = masi_distance(set(Reliability['X'][i]), set(Reliability['G'][i]))
#    print(x)

In [85]:
# Create dictionaries with frequencies of observation of each label per annotator

label_freqs_X = FreqDist(Reliability['X'])
label_freqs_G = FreqDist(Reliability['G'])


print('Here are the lable frequencies for X annotation:')
print(label_freqs_X.items())
print("\n")
print('Here are the lable frequencies for G annotation:')
print(label_freqs_G.items())

Here are the lable frequencies for X annotation:
dict_items([(('Lexicalized metaphor', nan, nan), 80), (('Humour', nan, nan), 1), ((nan, nan, nan), 50), (('Heuristic reasoning', nan, nan), 4), (('Visualization', nan, nan), 29), (('Visualization', 'Artistic metaphor', nan), 2), (('Persuasiveness', nan, nan), 10), (('Visualization', 'Persuasiveness', nan), 8), (('Persuasiveness', 'Heuristic reasoning', nan), 1), (('Visualization', 'Persuasiveness', 'Artistic metaphor'), 2), (('Persuasiveness', 'Artistic metaphor', 'Visualization'), 1), (('Visualization', 'Humour', nan), 3), (('Visualization', 'Artistic metaphor', 'Persuasiveness'), 1), (('Visualization', 'Explanation', nan), 2), (('Persuasiveness', 'Visualization', nan), 2), (('Persuasiveness', 'Argumentative metaphor', 'Visualization'), 1), (('Explanation', 'Visualization', nan), 1), (('Humour', 'Heuristic reasoning', nan), 1), (('Artistic metaphor', nan, nan), 1)])


Here are the lable frequencies for G annotation:
dict_items([(('Visua

In [90]:
# Merge "label_freqs_X" and "label_freqs_G" into a single dictionary "lable_freqs"

# Maybe I should first merge X and G and then turn it into FreqDist ? Seems to yeld the same result...
# F = np.concatenate([Reliability['X'], Reliability['G']])
# label_freqs_2 = FreqDist(F)

label_freqs = label_freqs_X + label_freqs_G


In [49]:
# Implementation of Krippendorff's alpha with MASI distance and 2 coders

# Adapted from NLTK library, based on '''Artstein and Poesio 2008'''

In [91]:
def Expected_disagreement(label_freqs):
    
    numer_units = sum(label_freqs.values())//2 # Number of annotated units
    total_de = 0.0
    for j, kj in label_freqs.items():
        for l, kl in label_freqs.items():
            total_de += float(kj * kl) * masi_distance(set(j), set(l)) # Here we need to convert to a set for MASI
    de = 1.0 * total_de / (numer_units*2 * (numer_units*2 - 1)) # CHECK here, Poesio *2 while nltk without
    return de

0.6547669152046789

In [92]:
def Observed_disagreement(Dataset, annotator_1, annotator_2, label_freqs):
    
        numer_units = sum(label_freqs.values())
        total_do = 0.0  
        for i in range(numer_units//2 - 1):
            for j, kj in label_freqs.items():
                for l, kl in label_freqs.items():
                    
                    if Dataset[annotator_1][i] == j and Dataset[annotator_2][i] == j:
                        nkj = 2
                    elif Dataset[annotator_1][i] == j and Dataset[annotator_2][i] != j:
                        nkj = 1
                    elif Dataset[annotator_1][i] != j and Dataset[annotator_2][i] == j:
                        nkj = 1
                    elif Dataset[annotator_1][i] != j and Dataset[annotator_2][i] != j:
                        nkj = 0
                        
                    if Dataset[annotator_1][i] == l and Dataset[annotator_2][i] == l:
                        nkl = 2
                    elif Dataset[annotator_1][i] == l and Dataset[annotator_2][i] != l:
                        nkl = 1
                    elif Dataset[annotator_1][i] != l and Dataset[annotator_2][i] == l:
                        nkl = 1
                    elif Dataset[annotator_1][i] != l and Dataset[annotator_2][i] != l:
                        nkl = 0
                    
                    total_do += float(nkj * nkl) * masi_distance(set(j), set(l))

        do = total_do / (numer_units * 2)
        return do

In [93]:
def alpha(Dataset, annotator_1, annotator_2, label_freqs):

        do = Observed_disagreement(Dataset, annotator_1, annotator_2, label_freqs)  # Observed disagreement
        de = Expected_disagreement(label_freqs)  # Expected disagreement
        k_alpha = 1.0 - do / de
        return k_alpha
    
alpha(Reliability, 'X', 'G', label_freqs)

0.6828610632099182